In [52]:
pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


Explore stock market dataset from Yahoo Finance

In [124]:
import yfinance as yf


In [125]:
spy = yf.download("SPY", period='max') # start="2017-01-01", end="2017-04-30")

[*********************100%%**********************]  4 of 4 completed


In [126]:
# SPY is proxy for SPX / S&P 500
spy.to_csv('data/spy.csv')

In [141]:
# SPX tracks S&P 500 index
spx = yf.download("^SPX", period='max') 
spx

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0
...,...,...,...,...,...,...
2023-12-29,4782.879883,4788.430176,4751.990234,4769.830078,4769.830078,3126060000
2024-01-02,4745.200195,4754.330078,4722.669922,4742.830078,4742.830078,3743050000
2024-01-03,4725.069824,4729.290039,4699.709961,4704.810059,4704.810059,3950760000


In [142]:
spx.to_csv('data/spx.csv')

In [143]:
# NDX tracks NASDAQ 100 index
ndx = yf.download("^NDX", period='max')
ndx

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1985-10-01,110.620003,112.160004,110.565002,112.139999,112.139999,153160000
1985-10-02,112.139999,112.540001,110.779999,110.824997,110.824997,164640000
1985-10-03,110.839996,111.184998,110.120003,110.870003,110.870003,147300000
1985-10-04,110.870003,110.870003,109.855003,110.074997,110.074997,147900000
1985-10-07,110.074997,110.135002,108.175003,108.199997,108.199997,128640000
...,...,...,...,...,...,...
2023-12-29,16902.439453,16919.509766,16757.890625,16825.929688,16825.929688,5441060000
2024-01-02,16667.300781,16686.599609,16453.150391,16543.939453,16543.939453,5791170000
2024-01-03,16413.259766,16471.080078,16352.139648,16368.490234,16368.490234,5449950000


In [144]:
ndx.to_csv('data/ndx.csv')

In [138]:
# VIX volatility index
vix = yf.download("^VIX", period='max')
vix

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1990-01-02,17.240000,17.240000,17.240000,17.240000,17.240000,0
1990-01-03,18.190001,18.190001,18.190001,18.190001,18.190001,0
1990-01-04,19.219999,19.219999,19.219999,19.219999,19.219999,0
1990-01-05,20.110001,20.110001,20.110001,20.110001,20.110001,0
1990-01-08,20.260000,20.260000,20.260000,20.260000,20.260000,0
...,...,...,...,...,...,...
2023-12-29,12.550000,13.190000,12.360000,12.450000,12.450000,0
2024-01-02,13.220000,14.230000,13.100000,13.200000,13.200000,0
2024-01-03,13.350000,14.220000,13.330000,14.040000,14.040000,0


In [139]:
vix.to_csv('data/vix.csv')

In [145]:
# RUT tracks Russell 2000 small&mid caps index
rut = yf.download("^RUT", period='max')
rut

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1987-09-10,167.440002,168.979996,167.440002,168.970001,168.970001,179800000
1987-09-11,168.970001,170.539993,168.750000,170.539993,170.539993,178000000
1987-09-14,170.529999,170.949997,170.149994,170.429993,170.429993,154400000
1987-09-15,170.419998,170.440002,169.130005,169.199997,169.199997,136200000
1987-09-16,169.309998,170.029999,168.820007,168.919998,168.919998,195700000
...,...,...,...,...,...,...
2023-12-29,2054.229980,2056.620117,2027.069946,2027.069946,2027.069946,3126060000
2024-01-02,2012.750000,2032.489990,2003.410034,2012.800049,2012.800049,3743050000
2024-01-03,1995.170044,1995.170044,1958.030029,1959.199951,1959.199951,3950760000


In [146]:
rut.to_csv('data/rut.csv')

In [134]:
# USD index
dxy = yf.download('DX-Y.NYB', period='max')
dxy

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1971-01-04,120.529999,120.529999,120.529999,120.529999,120.529999,0
1971-01-05,120.519997,120.519997,120.519997,120.519997,120.519997,0
1971-01-06,120.489998,120.489998,120.489998,120.489998,120.489998,0
1971-01-07,120.550003,120.550003,120.550003,120.550003,120.550003,0
1971-01-08,120.529999,120.529999,120.529999,120.529999,120.529999,0
...,...,...,...,...,...,...
2023-12-29,101.199997,101.419998,101.070000,101.330002,101.330002,0
2024-01-02,101.419998,102.250000,101.339996,102.199997,102.199997,0
2024-01-03,102.150002,102.730003,102.080002,102.459999,102.459999,0


In [135]:
dxy.to_csv('data/dxy.csv')

In [136]:
# US Treasury 10 year interest rate
tnx = yf.download('^TNX', period='max')
tnx

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1962-01-02,4.060,4.060,4.060,4.060,4.060,0
1962-01-03,4.030,4.030,4.030,4.030,4.030,0
1962-01-04,3.990,3.990,3.990,3.990,3.990,0
1962-01-05,4.020,4.020,4.020,4.020,4.020,0
1962-01-08,4.030,4.030,4.030,4.030,4.030,0
...,...,...,...,...,...,...
2023-12-29,3.879,3.898,3.847,3.866,3.866,0
2024-01-02,3.969,3.973,3.918,3.946,3.946,0
2024-01-03,3.978,4.008,3.894,3.907,3.907,0


In [137]:
tnx.to_csv('data/tnx.csv')

In [ ]:

msft = yf.Ticker("MSFT")
print(msft)
"""
returns
<yfinance.Ticker object at 0x1a1715e898>
"""


In [54]:
import json

# get stock info
print(json.dumps(msft.info, indent=2))

"""
returns:
{
 'quoteType': 'EQUITY',
 'quoteSourceName': 'Nasdaq Real Time Price',
 'currency': 'USD',
 'shortName': 'Microsoft Corporation',
 'exchangeTimezoneName': 'America/New_York',
  ...
 'symbol': 'MSFT'
}
"""


{
  "address1": "One Microsoft Way",
  "city": "Redmond",
  "state": "WA",
  "zip": "98052-6399",
  "country": "United States",
  "phone": "425 882 8080",
  "website": "https://www.microsoft.com",
  "industry": "Software - Infrastructure",
  "industryKey": "software-infrastructure",
  "industryDisp": "Software - Infrastructure",
  "sector": "Technology",
  "sectorKey": "technology",
  "sectorDisp": "Technology",
  "longBusinessSummary": "Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across 

"\nreturns:\n{\n 'quoteType': 'EQUITY',\n 'quoteSourceName': 'Nasdaq Real Time Price',\n 'currency': 'USD',\n 'shortName': 'Microsoft Corporation',\n 'exchangeTimezoneName': 'America/New_York',\n  ...\n 'symbol': 'MSFT'\n}\n"

In [19]:

# get historical market data
msft_hist = msft.history(period="max")
print(msft_hist)
"""
returns:
              Open    High    Low    Close      Volume  Dividends  Splits
Date
1986-03-13    0.06    0.07    0.06    0.07  1031788800        0.0     0.0
1986-03-14    0.07    0.07    0.07    0.07   308160000        0.0     0.0
...
2019-04-15  120.94  121.58  120.57  121.05    15792600        0.0     0.0
2019-04-16  121.64  121.65  120.10  120.77    14059700        0.0     0.0
"""


                                 Open        High         Low       Close  \
Date                                                                        
1986-03-13 00:00:00-05:00    0.054893    0.062965    0.054893    0.060274   
1986-03-14 00:00:00-05:00    0.060274    0.063504    0.060274    0.062427   
1986-03-17 00:00:00-05:00    0.062427    0.064042    0.062427    0.063504   
1986-03-18 00:00:00-05:00    0.063504    0.064042    0.061350    0.061889   
1986-03-19 00:00:00-05:00    0.061889    0.062427    0.060274    0.060812   
...                               ...         ...         ...         ...   
2023-12-29 00:00:00-05:00  376.000000  377.160004  373.480011  376.040009   
2024-01-02 00:00:00-05:00  373.859985  375.899994  366.769989  370.869995   
2024-01-03 00:00:00-05:00  369.010010  373.260010  368.510010  370.600006   
2024-01-04 00:00:00-05:00  370.670013  373.100006  367.170013  367.940002   
2024-01-05 00:00:00-05:00  368.970001  372.059998  366.500000  367.750000   

'\nreturns:\n              Open    High    Low    Close      Volume  Dividends  Splits\nDate\n1986-03-13    0.06    0.07    0.06    0.07  1031788800        0.0     0.0\n1986-03-14    0.07    0.07    0.07    0.07   308160000        0.0     0.0\n...\n2019-04-15  120.94  121.58  120.57  121.05    15792600        0.0     0.0\n2019-04-16  121.64  121.65  120.10  120.77    14059700        0.0     0.0\n'

In [55]:
# show actions (dividends, splits)
print(msft.actions)
"""
returns:
            Dividends  Splits
Date
1987-09-21       0.00     2.0
1990-04-16       0.00     2.0
...
2018-11-14       0.46     0.0
2019-02-20       0.46     0.0
"""

                           Dividends  Stock Splits
Date                                              
1987-09-21 00:00:00-04:00       0.00           2.0
1990-04-16 00:00:00-04:00       0.00           2.0
1991-06-27 00:00:00-04:00       0.00           1.5
1992-06-15 00:00:00-04:00       0.00           1.5
1994-05-23 00:00:00-04:00       0.00           2.0
...                              ...           ...
2022-11-16 00:00:00-05:00       0.68           0.0
2023-02-15 00:00:00-05:00       0.68           0.0
2023-05-17 00:00:00-04:00       0.68           0.0
2023-08-16 00:00:00-04:00       0.68           0.0
2023-11-15 00:00:00-05:00       0.75           0.0

[89 rows x 2 columns]


'\nreturns:\n            Dividends  Splits\nDate\n1987-09-21       0.00     2.0\n1990-04-16       0.00     2.0\n...\n2018-11-14       0.46     0.0\n2019-02-20       0.46     0.0\n'

In [56]:

# show dividends
print(msft.dividends)
"""
returns:
Date
2003-02-19    0.08
2003-10-15    0.16
...
2018-11-14    0.46
2019-02-20    0.46
"""

Date
2003-02-19 00:00:00-05:00    0.08
2003-10-15 00:00:00-04:00    0.16
2004-08-23 00:00:00-04:00    0.08
2004-11-15 00:00:00-05:00    3.08
2005-02-15 00:00:00-05:00    0.08
                             ... 
2022-11-16 00:00:00-05:00    0.68
2023-02-15 00:00:00-05:00    0.68
2023-05-17 00:00:00-04:00    0.68
2023-08-16 00:00:00-04:00    0.68
2023-11-15 00:00:00-05:00    0.75
Name: Dividends, Length: 80, dtype: float64


'\nreturns:\nDate\n2003-02-19    0.08\n2003-10-15    0.16\n...\n2018-11-14    0.46\n2019-02-20    0.46\n'

In [57]:
# show splits
print(msft.splits)
"""
returns:
Date
1987-09-21    2.0
1990-04-16    2.0
...
1999-03-29    2.0
2003-02-18    2.0
"""

Date
1987-09-21 00:00:00-04:00    2.0
1990-04-16 00:00:00-04:00    2.0
1991-06-27 00:00:00-04:00    1.5
1992-06-15 00:00:00-04:00    1.5
1994-05-23 00:00:00-04:00    2.0
1996-12-09 00:00:00-05:00    2.0
1998-02-23 00:00:00-05:00    2.0
1999-03-29 00:00:00-05:00    2.0
2003-02-18 00:00:00-05:00    2.0
Name: Stock Splits, dtype: float64


'\nreturns:\nDate\n1987-09-21    2.0\n1990-04-16    2.0\n...\n1999-03-29    2.0\n2003-02-18    2.0\n'

In [58]:
msft_hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 05:00:00+00:00,0.054893,0.062965,0.054893,0.060274,1031788800,0.0,0.0
1986-03-14 05:00:00+00:00,0.060274,0.063504,0.060274,0.062427,308160000,0.0,0.0
1986-03-17 05:00:00+00:00,0.062427,0.064042,0.062427,0.063504,133171200,0.0,0.0
1986-03-18 05:00:00+00:00,0.063504,0.064042,0.061350,0.061889,67766400,0.0,0.0
1986-03-19 05:00:00+00:00,0.061889,0.062427,0.060274,0.060812,47894400,0.0,0.0


In [59]:
msft_hist.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-12-29 05:00:00+00:00,376.000000,377.160004,373.480011,376.040009,18723000,0.0,0.0
2024-01-02 05:00:00+00:00,373.859985,375.899994,366.769989,370.869995,25258600,0.0,0.0
2024-01-03 05:00:00+00:00,369.010010,373.260010,368.510010,370.600006,23083500,0.0,0.0
2024-01-04 05:00:00+00:00,370.670013,373.100006,367.170013,367.940002,20901500,0.0,0.0
2024-01-05 05:00:00+00:00,368.970001,372.059998,366.500000,367.750000,20987000,0.0,0.0


In [104]:
# create a deep copy so we can experiment with data without triggering unnecessary downloads 
df=msft_hist.copy(deep=True)

# df=msft_hist.reset_index(drop=True)
# data['Date']=pd.to_datetime(data['Date'])
print(len(df))
df.head()

9531


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 05:00:00+00:00,0.054893,0.062965,0.054893,0.060274,1031788800,0.0,0.0
1986-03-14 05:00:00+00:00,0.060274,0.063504,0.060274,0.062427,308160000,0.0,0.0
1986-03-17 05:00:00+00:00,0.062427,0.064042,0.062427,0.063504,133171200,0.0,0.0
1986-03-18 05:00:00+00:00,0.063504,0.064042,0.061350,0.061889,67766400,0.0,0.0
1986-03-19 05:00:00+00:00,0.061889,0.062427,0.060274,0.060812,47894400,0.0,0.0


Ingest data into darts timeseries


In [105]:
# Count rows without values
# https://stackoverflow.com/questions/28199524/best-way-to-count-the-number-of-rows-with-missing-values-in-a-pandas-dataframe
df.shape[0] - df.dropna().shape[0]



0

In [106]:
# count any cells without values
df.isnull().values.ravel().sum()

0

Convert data to timeseries format that models can work with

Regular time intervals between data points and no missing values

In [107]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 05:00:00+00:00,0.054893,0.062965,0.054893,0.060274,1031788800,0.0,0.0
1986-03-14 05:00:00+00:00,0.060274,0.063504,0.060274,0.062427,308160000,0.0,0.0
1986-03-17 05:00:00+00:00,0.062427,0.064042,0.062427,0.063504,133171200,0.0,0.0
1986-03-18 05:00:00+00:00,0.063504,0.064042,0.061350,0.061889,67766400,0.0,0.0
1986-03-19 05:00:00+00:00,0.061889,0.062427,0.060274,0.060812,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2023-12-29 05:00:00+00:00,376.000000,377.160004,373.480011,376.040009,18723000,0.0,0.0
2024-01-02 05:00:00+00:00,373.859985,375.899994,366.769989,370.869995,25258600,0.0,0.0
2024-01-03 05:00:00+00:00,369.010010,373.260010,368.510010,370.600006,23083500,0.0,0.0


In [108]:
import pandas as pd

# Clean up date index. Remove time zone.
df.index = pd.to_datetime(df.index, utc=True).date
df.index = pd.to_datetime(df.index, utc=True)
df.index.name = 'Date'
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00+00:00,0.054893,0.062965,0.054893,0.060274,1031788800,0.0,0.0
1986-03-14 00:00:00+00:00,0.060274,0.063504,0.060274,0.062427,308160000,0.0,0.0
1986-03-17 00:00:00+00:00,0.062427,0.064042,0.062427,0.063504,133171200,0.0,0.0
1986-03-18 00:00:00+00:00,0.063504,0.064042,0.061350,0.061889,67766400,0.0,0.0
1986-03-19 00:00:00+00:00,0.061889,0.062427,0.060274,0.060812,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2023-12-29 00:00:00+00:00,376.000000,377.160004,373.480011,376.040009,18723000,0.0,0.0
2024-01-02 00:00:00+00:00,373.859985,375.899994,366.769989,370.869995,25258600,0.0,0.0
2024-01-03 00:00:00+00:00,369.010010,373.260010,368.510010,370.600006,23083500,0.0,0.0


In [109]:
df.index

DatetimeIndex(['1986-03-13 00:00:00+00:00', '1986-03-14 00:00:00+00:00',
               '1986-03-17 00:00:00+00:00', '1986-03-18 00:00:00+00:00',
               '1986-03-19 00:00:00+00:00', '1986-03-20 00:00:00+00:00',
               '1986-03-21 00:00:00+00:00', '1986-03-24 00:00:00+00:00',
               '1986-03-25 00:00:00+00:00', '1986-03-26 00:00:00+00:00',
               ...
               '2023-12-21 00:00:00+00:00', '2023-12-22 00:00:00+00:00',
               '2023-12-26 00:00:00+00:00', '2023-12-27 00:00:00+00:00',
               '2023-12-28 00:00:00+00:00', '2023-12-29 00:00:00+00:00',
               '2024-01-02 00:00:00+00:00', '2024-01-03 00:00:00+00:00',
               '2024-01-04 00:00:00+00:00', '2024-01-05 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=9531, freq=None)

In [110]:
type(df.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [111]:
from darts import TimeSeries

series = TimeSeries.from_dataframe(df, fill_missing_dates=True, freq='B') # value_cols=["Close"],  "Open", "High", "Low", "Close", "Volume", "Dividends", "Stock Splits"


The provided DatetimeIndex was associated with a timezone, which is currently not supported by xarray. To avoid unexpected behaviour, the tz information was removed. Consider calling `ts.time_index.tz_localize(UTC)` when exporting the results.To plot the series with the right time steps, consider setting the matplotlib.pyplot `rcParams['timezone']` parameter to automatically convert the time axis back to the original timezone.


In [112]:
# get regularized time series with NaN fill-ins
reg_df = series.pd_dataframe()

In [113]:
# get number of rows without values
reg_df.shape[0] - reg_df.dropna().shape[0]

336

In [115]:
# fill in missing values
reg_df = reg_df.interpolate()


In [116]:
# Check again number of rows without values. Should be 0.
reg_df.shape[0] - reg_df.dropna().shape[0]

0

In [117]:
# update series with new regularized dates and values
series = TimeSeries.from_dataframe(reg_df)

Save prepared timeseries data to local csv for model training

In [118]:
data_file_name = 'data/market_data.csv'

In [119]:
# series.to_csv(data_file_name)
series.to_csv(data_file_name)

Make sure data can load back into timeseries

In [120]:
series = TimeSeries.from_csv(data_file_name, time_col='Date') #, freq='B')

In [121]:
series

<TimeSeries (DataArray) (Date: 9867, component: 7, sample: 1)>
array([[[5.48928341e-02],
        [6.29653825e-02],
        [5.48928341e-02],
        ...,
        [1.03178880e+09],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[6.02741448e-02],
        [6.35035369e-02],
        [6.02741448e-02],
        ...,
        [3.08160000e+08],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[6.24266277e-02],
        [6.40416324e-02],
        [6.24266277e-02],
        ...,
...
        ...,
        [2.30835000e+07],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[3.70670013e+02],
        [3.73100006e+02],
        [3.67170013e+02],
        ...,
        [2.09015000e+07],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[3.68970001e+02],
        [3.72059998e+02],
        [3.66500000e+02],
        ...,
        [2.09870000e+07],
        [0.00000000e+00],
        [0.00000000e+00]]])
Coordinates:
  * Date       (Date) datetime64[ns] 1986-03-13 1986-03-14 ... 2024-01-05
  * component  (component) object 'Open' 'High' ... 'Dividends' 'Stock Splits'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [122]:
# get regularized time series with NaN fill-ins
loaded_df = series.pd_dataframe()

In [123]:
# get number of rows without values
loaded_df.shape[0] - loaded_df.dropna().shape[0]

0